In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sn
sn.set_context("poster")

import torch
from torch import nn as nn
ttype = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor
ctype = torch.cuda.LongTensor if torch.cuda.is_available() else torch.LongTensor
print(ttype)
import torch.nn.functional as F
from matplotlib import gridspec
from sithcon import SITHCon_Layer, _SITHCon_Core, iSITH

from tqdm.notebook import tqdm

import itertools
from csv import DictWriter
import os 
from os.path import join
import glob

import numpy as np
import pandas as pd
import pickle
from math import factorial
import random

In [ ]:
full_path = join('/media', 'root', 'easystore', 'experiments', 
                 'audiomnist', '01.00', 'AudioMNIST', 'data')

In [ ]:
all_files = pd.read_csv(join('data','files_info.csv'))
all_dat = []
for f in glob.glob(join(full_path, "*")):
    print(f)
    all_dat.append(np.load(join(full_path, f)))
all_dat = np.concatenate(all_dat, axis=0)

In [ ]:
all_files['ind_idx'] = all_files.dat_idx[:]
all_files.dat_idx =  (all_files.subjid - 1)*500 + all_files.dat_idx

In [ ]:
trainX = all_dat[list(all_files.loc[(all_files.rec_split<9)&(all_files.subjid!=60), 'dat_idx'])]
trainY = np.array(all_files.loc[(all_files.rec_split<9)&(all_files.subjid!=60), 'digit'])
testX = all_dat[list(all_files.loc[(all_files.rec_split==9)&(all_files.subjid!=60), 'dat_idx'])]
testY = np.array(all_files.loc[(all_files.rec_split==9)&(all_files.subjid!=60), 'digit'])
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

In [ ]:
batch_size = 16
dataset = torch.utils.data.TensorDataset(torch.Tensor(trainX).cuda(), 
                                         torch.LongTensor(trainY).cuda())
dataset = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

dataset_valid = torch.utils.data.TensorDataset(torch.Tensor(testX).cuda(), 
                                               torch.LongTensor(testY).cuda())
dataset_valid = torch.utils.data.DataLoader(dataset_valid, batch_size=batch_size, shuffle=False)

# Classes 

In [ ]:
import torch
import torch.nn as nn
from torch.nn.utils import weight_norm


class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()


class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2):
        super(TemporalBlock, self).__init__()
        self.conv1 = weight_norm(nn.Conv1d(n_inputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp1 = Chomp1d(padding)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        self.conv2 = weight_norm(nn.Conv1d(n_outputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp2 = Chomp1d(padding)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)

        self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.dropout1,
                                 self.conv2, self.chomp2, self.relu2, self.dropout2)
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()
        self.init_weights()

    def init_weights(self):
        self.conv1.weight.data.normal_(0, 0.01)
        self.conv2.weight.data.normal_(0, 0.01)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x):
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)


class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size, dropout=dropout)]

        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)
class TCN(nn.Module):
    def __init__(self, input_size, output_size, num_channels, kernel_size, dropout):
        super(TCN, self).__init__()
        self.tcn = TemporalConvNet(input_size, num_channels, kernel_size=kernel_size, dropout=dropout)
        self.linear = nn.Linear(num_channels[-1], output_size)
        self.init_weights()

    def init_weights(self):
        self.linear.weight.data.normal_(0, 0.01)

    def forward(self, x):
        y1 = self.tcn(x)
        return self.linear(y1[:, :, -1])

# Functions

In [ ]:
def gen_model(p):
    model = TCN(50, 10, [25]*8, kernel_size=16, dropout=0.0).cuda()
    return model

def train(model, ttype, train_loader, test_loader, optimizer, loss_func, epoch, perf_file,
          loss_buffer_size=100, batch_size=4, device='cuda',
          prog_bar=None, last_test_perf=0):
    
    assert(loss_buffer_size%batch_size==0)
    perfs = []
    losses = []
    best_test_perf = last_test_perf
    
    for batch_idx, (data, target) in enumerate(train_loader):
        model.train()
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = loss_func(out,
                         target)

        loss.backward()
        optimizer.step()
        perfs.append((torch.argmax(out, dim=-1) == 
                      target).sum().item())
        perfs = perfs[int(-loss_buffer_size/batch_size):]
                          
        losses.append(loss.detach().cpu().numpy())
        losses = losses[int(-loss_buffer_size/batch_size):]
        
        if ((batch_idx*batch_size)%loss_buffer_size == 0) & (batch_idx != 0):
            loss_track = {}
            last_test_perf = np.mean(perfs) / (batch_size)
            #last_test_perf = test_model(model, 'cuda', test_loader, 
            #                            batch_size)
            loss_track['avg_loss'] = np.mean(losses)
            #loss_track['last_test'] = last_test_perf
            loss_track['training_perf'] = np.mean(perfs) / (batch_size)
            loss_track['epoch'] = epoch
            loss_track['batch_idx'] = batch_idx
            with open(perf_file, 'a+') as fp:
                csv_writer = DictWriter(fp, fieldnames=list(loss_track.keys()))
                if fp.tell() == 0:
                    csv_writer.writeheader()
                csv_writer.writerow(loss_track)
                fp.flush()
            if best_test_perf < last_test_perf:
                torch.save(model.state_dict(), perf_file[:-4]+".pt")
                best_test_perf = last_test_perf
        if not (prog_bar is None):
            # Update progress_bar
            s = "{}:{} Loss: {:.5f}, Tperf: {:.4f}, valid: {:.4f}"
            format_list = [epoch,batch_idx*batch_size, np.mean(losses), 
                           np.mean(perfs) / (batch_size),
                           last_test_perf]         
            s = s.format(*format_list)
            prog_bar.set_description(s)
    return best_test_perf
def test_model(model, device, test_loader, batch_size):
    # Test the Model
    perfs = []
    tot = 0.0
    total_num = 0.0
    with torch.no_grad():
        for data, target in test_loader:
            data = data.to(device)
            batch = data.shape[0]
            target = target.to(device)
            out = model(data)
            total_num += batch
            tot += (torch.argmax(out, 
                                 dim=-1) == target).sum().item()
            
    perf = tot / total_num
    return perf

def save_outcome(outcome, filename):
    dat = pd.DataFrame(outcome)
    dat.to_csv(join('perf',filename))

# Go!

In [ ]:
params = [
          [None],
          ]


In [ ]:
runs = 5
for r in range(runs):
    for i, p in enumerate(params):
        model = gen_model(p)

        tot_weights = 0
        for p in model.parameters():
            tot_weights += p.numel()
        print("Total Weights:", tot_weights)

        loss_func = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters())
        epochs = 10
        Trainscale = 10
        device='cuda'
        progress_bar = tqdm(range(int(epochs)), bar_format='{l_bar}{bar:5}{r_bar}{bar:-5b}')
        epochperfs = []
        times_100 = 0
        last_test = 0
        for epoch_idx in progress_bar:
            perfs = []
            losses = []
            model.train()
            last_test = train(model, ttype, dataset, dataset_valid, 
                              optimizer, loss_func, batch_size=batch_size, loss_buffer_size=992,
                              epoch=epoch_idx, perf_file=join('perf','tcn_audioHARD_05232021_{}.csv'.format(r)),
                              prog_bar=progress_bar, last_test_perf=last_test)
            

In [ ]:
model.load_state_dict(torch.load('perf/tcn_audioHARD_5032021_0.pt'))

In [ ]:
for r in range(5):
    model.load_state_dict(torch.load('perf/tcn_audioHARD_05232021_{}.pt'.format(r)))
    model.eval()
    scales = [
              '10.00', 
              '05.00', 
              '02.50',
              '01.25', 
              '01.00', 
              '00.80', 
              '00.40', 
              '00.20', 
              '00.10'
              ]
    scale_perf = []
    for scale in scales:
        test_paths = join('/media', 'root', 'easystore', 'experiments', 
                         'audiomnist', scale, 'AudioMNIST', 'data')
        full_file = glob.glob(join(test_paths, "*"))
        subj_perfs = []
        for filename in full_file:
            test_dat = np.load(filename)

            testX = test_dat[list(all_files.loc[(all_files.rec_split==9)&
                                                (all_files.subjid==int(filename.split("_")[-2])), 'ind_idx'])]
            testY = np.array(all_files.loc[(all_files.rec_split==9)&
                                           (all_files.subjid==int(filename.split("_")[-2])), 'digit'])
            dataset_test = torch.utils.data.TensorDataset(torch.Tensor(testX).cuda(), 
                                                           torch.LongTensor(testY).cuda())
            dataset_test = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size, shuffle=False)
            last_test_perf = test_model(model, 'cuda',
                                        dataset_test, batch_size)
            subj_perfs.append(last_test_perf)
        scoredict = {'perf':np.mean(subj_perfs),
                           'scal':float(scale)}
        print(scoredict)
        scale_perf.append(scoredict)
    scale_perfs = pd.DataFrame(scale_perf)
    scale_perfs.to_pickle(join("perf", "tcn_hard_test_{}.dill".format(r)))